In [ ]:
# 필요한 패키지 불러오기
import os
import torch
from torch import nn, optim
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from PIL import Image
import pandas as pd
import torch.nn.functional as F
from tqdm import tqdm  # 학습 진행률 표시

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("사용 중인 디바이스:", device)

## 1. 데이터 준비 및 전처리

In [ ]:
# 이미지 전처리 정의 (ConvNeXt 권장 크기: 224x224)
transform = transforms.Compose([
    transforms.Resize((224, 224)),            # ConvNeXt 입력 사이즈로 맞춤
    transforms.ToTensor(),                    # PIL -> 텐서 변환
    transforms.Normalize([0.5]*3, [0.5]*3)    # RGB 정규화 (중앙:0, 범위: -1~1)
])

# 데이터 경로 설정
data_dir = "/home/ubuntu/Dacon/datasets/augmentated_data_4split"
test_dir = "/home/ubuntu/Dacon/datasets/test"

# 데이터셋 로드
dataset = ImageFolder(data_dir, transform=transform)

# train:val - 8:2 비율로 분할
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 배치 단위로 데이터 로딩
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 클래스(라벨) 이름 확인
class_names = dataset.classes
num_classes = len(class_names)
print(f"클래스 목록: {class_names}")
print(f"클래스 수: {num_classes}")

## 2. ConvNeXt Tiny 모델 정의

In [ ]:
# ConvNeXt Tiny 모델 불러오기 (사전학습된 가중치 사용)
model = models.convnext_tiny(pretrained=True)

# 분류기 마지막 층을 클래스 수에 맞게 변경
model.classifier[2] = nn.Linear(model.classifier[2].in_features, num_classes)

# GPU로 모델 이동
model = model.to(device)

# 손실 함수: 다중 클래스 분류를 위한 Cross Entropy Loss
criterion = nn.CrossEntropyLoss()

# 옵티마이저: Adam (학습률 1e-4)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

## 3. 모델 평가 함수

In [ ]:
# 검증 함수 정의
def evaluate(model, val_loader):
    model.eval()  # 평가 모드 (Dropout, BatchNorm 등 비활성화)
    correct = 0
    val_loss = 0
    
    with torch.no_grad():  # 그래디언트 계산 비활성화
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_loss += criterion(outputs, labels).item()
            correct += (outputs.argmax(1) == labels).sum().item()
    
    val_loss /= len(val_loader)
    accuracy = correct / len(val_loader.dataset)
    print(f"검증 손실: {val_loss:.4f}, 정확도: {accuracy:.4f}")
    return accuracy

## 4. 모델 학습

In [ ]:
# 모델 학습 함수 정의
def train_model(model, train_loader, val_loader, epochs=10):
    best_acc = 0.0
    best_model_path = "jhs_convnext_best.pth"  # 최고 성능 모델 저장 경로
    
    for epoch in range(epochs):
        print(f"\n===== Epoch {epoch+1}/{epochs} =====\n")
        model.train()  # 학습 모드 설정
        train_loss = 0
        train_correct = 0
        
        # 학습 루프
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()               # 그래디언트 초기화
            outputs = model(images)             # 모델 예측
            loss = criterion(outputs, labels)   # 손실 계산
            loss.backward()                     # 역전파
            optimizer.step()                    # 가중치 업데이트
            
            train_loss += loss.item()
            train_correct += (outputs.argmax(1) == labels).sum().item()
        
        # 학습 손실 및 정확도 계산
        train_loss /= len(train_loader)
        train_acc = train_correct / len(train_loader.dataset)
        print(f"[{epoch+1}] 학습 손실: {train_loss:.4f}, 정확도: {train_acc:.4f}")
        
        # 검증 데이터로 정확도 평가
        val_acc = evaluate(model, val_loader)
        
        # 최고 성능 모델 저장
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), best_model_path)
            print(f"새로운 최고 모델 저장! 검증 정확도: {best_acc:.4f}")
    
    # 마지막 모델도 저장
    torch.save(model.state_dict(), "jhs_car_model_convnext.pth")
    print(f"최종 모델 저장 완료")
    print(f"최고 성능 모델 정확도: {best_acc:.4f} (경로: {best_model_path})")
    
    # 최고 성능 모델 복원
    model.load_state_dict(torch.load(best_model_path))
    return model

In [ ]:
# 모델 학습 실행 (10 에포크)
model = train_model(model, train_loader, val_loader, epochs=10)

## 5. 추론 및 제출 파일 생성

In [ ]:
# 저장된 모델 불러오기
model = models.convnext_tiny(pretrained=False)
model.classifier[2] = nn.Linear(model.classifier[2].in_features, num_classes)
model.load_state_dict(torch.load("jhs_car_model_convnext.pth", map_location=device))
model.to(device)
model.eval()

In [ ]:
# 제출용 파일 양식 불러오기
submission_path = "/home/ubuntu/Dacon/result/sample_submission.csv"
submission = pd.read_csv(submission_path, encoding='utf-8-sig')
class_names = submission.columns[1:]  # 'ID' 제외한 클래스 컬럼명

In [ ]:
# 테스트 이미지 목록 가져오기
image_files = sorted([f for f in os.listdir(test_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
print(f"테스트 이미지 수: {len(image_files)}개")

In [ ]:
# 추론 실행
results = []

with torch.no_grad():
    for fname in tqdm(image_files, desc="테스트 이미지 예측 중"):
        image_path = os.path.join(test_dir, fname)
        img = Image.open(image_path).convert("RGB")
        x = transform(img).unsqueeze(0).to(device)
        
        logits = model(x)
        probs = F.softmax(logits, dim=1).squeeze().cpu().tolist()
        
        results.append([fname] + probs)

In [ ]:
# 결과를 DataFrame으로 변환
pred = pd.DataFrame(results, columns=["ID"] + list(class_names))

# 제출 파일 생성
submission[class_names] = pred[class_names].values
submission.to_csv("./jhs_convnext_submission.csv", index=False, encoding='utf-8-sig')``

In [ ]:
# 제출 파일 확인
submission.head()